We're covering convolutions in my Mathematical Methods of Physics class now, and I've never loved any of the textbook explanations I've seen. I especially dislike the ones that obscure the underlying physical interpretation, and [A Student's Guide to Fourier Transforms](http://www.cambridge.org/us/academic/subjects/physics/mathematical-methods/students-guide-fourier-transforms-applications-physics-and-engineering-3rd-edition?format=PB&isbn=9780521176835#pjGd6C740q0xyglZ.97) by James is definitely the best I've seen. Here's my version of his version.

The goal is to get to understanding the definition
$$
C(x) = \int_{-\infty}^\infty F_1(x')F_2(x-x')dx'
$$
Where $C$ is the convolution of the two functions $F_1$ and $F_2$. Or, equivalently, we'd like to understand the more physically-motivated 
$$
\mathcal{O}(\lambda) = \int_{-\infty}^\infty S(\lambda_1)I(\lambda_1 - \lambda)d\lambda_1
$$
where we have some input signal $S$ which we're recording on an instrument that has the "instrument function"<sup name="a1">[1](#f1)</sup> $I$ and which, given that input signal, produces an output signal $\mathcal{O}$.



Perhaps surprisingly, we're going to start with some basic calculus as a reminder, and work our way up. Also, because it's the order we usually cover things in class, we'll start with a Fourier transform.

<b id="f1">1</b> also called an "instrument response function" or an "instrumental function" or an "impulse response" or a "point-spread function" or even a "Green's function" [↩](#a1)

A Gaussian of width $a$ is defined as $\frac{1}{a\sqrt{2\pi}}e^{-x^2/a^2}$ Let's plot it:

**NOTE** To make this easier to read, I've moved a lot of the code to the bottom. If you're running this as an interactive notebook, run the code at the bottom first!

In [12]:
def gaussian(x,a):
    return np.exp(-(x**2)/(a**2))/(a*np.sqrt(2*np.pi))
def plotgaussian(a=1):
    x = np.linspace(-10,10,1000)
    plt.plot(x,gaussian(x,a))
interact(plotgaussian,a=(0.1,10,0.1));

interactive(children=(FloatSlider(value=1.0, description='a', max=10.0, min=0.1), Output()), _dom_classes=('widget-interact',))

In class, we used a very clever trick<sup name="a2">[2](#f2)</sup> to show that the Fourier transform of a wide gaussian is a narrow gaussian, and vice versa. We can look at this in a couple of ways, but first, let's just make Python plot this for us.

`np.fft` will do a (fast) Fourier transform (FFT) for us. Using `np.fft` requires (unfortunately) knowing a couple of things. First, there's a call to `np.fft.fftshift` because `numpy` follows the seemingly standard convention that the results of a Fourier transform come in an odd order. `fftshift` shifts the frequencies back into the order you expect. Second, the resutls of an FFT are not normalized the way you'd expect; setting `norm='ortho'` fixes that.

<b id="f2">2</b> to jog your memory, it involved polar coordinates, and a square root, and was very clever. [↩](#a2)

In [34]:
def plotgaussian_pair(a=0.5):
    x = np.linspace(-10,10,1000)
    y = gaussian(x,a)
    y_fft = np.fft.fftshift(np.abs(np.fft.fft(y,norm='ortho')))
    plt.plot(x,y,label=r'$f(x)$')
    plt.plot(x,y_fft,label=r'$g(\alpha)$')
    plt.legend()
interact(plotgaussian_pair,a=(0.01,1.5,0.01));

interactive(children=(FloatSlider(value=0.5, description='a', max=1.5, min=0.01, step=0.01), Output()), _dom_classes=('widget-interact',))

Great. So, we can now see that numerical results agree with what we showed in class. Now let's figure out how to calculate it ourselves.

We'll pick the following definition of a Fourier Transform: given a function $f(x)$, define it's Fourier transform to be
$$
g(\alpha) = \int_{-\infty}^{\infty}f(x)e^{2\pi i x \alpha}dx
$$
and I'd like to do a bit of an unusual thing: I'd like to break that integral down piece by piece. Let's make life a little bit easy, and say $a=1$. Then, for my gaussian, I want
$$
g(\alpha) = \int_{-\infty}^{\infty}\frac{1}{\sqrt{2\pi}}e^{-x^2}e^{2\pi i x \alpha}dx
$$


Let's say I want to know what the frequency component at a specific $\alpha$ is. If I just want a numerical answer, I can use `scipy.integrate` to figure out the answer. For instance, if I want $g(0)$, I could evaluate
$$
g(\alpha) = \int_{-\infty}^{\infty}\frac{1}{\sqrt{2\pi}}e^{-x^2}dx
$$
We did this analytically in class, but numerically, it looks like

In [39]:
def f(x):
    return np.exp(-x**2)/np.sqrt(2*np.pi)
sp.integrate.quad(f,-1000,1000)

(0.7071067811865475, 3.099087934085691e-09)

In [49]:
def g(alpha):
    def _f(x):
        return np.real(np.exp(-x**2)*np.exp(2*np.pi*1j*alpha)/np.sqrt(2*np.pi))
    return sp.integrate.quad(_f,-1000,1000)

In [52]:
g(np.array([1,2,3]))

TypeError: only length-1 arrays can be converted to Python scalars

In [77]:
def plotgaussian_pair2(a=0.5):
    x = np.linspace(-10,10,1000)
    y = gaussian(x,a)
    def g(alpha):
        def _f(x):
            return np.real(np.exp(-x**2)*np.exp(2*np.pi*1j*alpha*a)/np.sqrt(2*np.pi))
        return sp.integrate.quad(_f,-10,10)[0]

    y_fft = np.array([g(alpha) for alpha in x])
    print(y_fft.shape)
    plt.plot(x,y,label=r'$f(x)s$')
    plt.plot(x,y_fft,label=r'$g(\alpha)r$')
    plt.ylim(-2,2)
    plt.legend()
interact(plotgaussian_pair2,a=(0.01,1.5,0.01));

interactive(children=(FloatSlider(value=0.5, description='a', max=1.5, min=0.01, step=0.01), Output()), _dom_classes=('widget-interact',))

Fuck everyone. What I really want to say is to recap the bit that

> an infinitesimal interval of the spectrum can be considered as a monochromatic line at $\lambda_1$, say, and of intensity $S(\lambda_1)d\lambda_1$, and is plotted by the spectrometer as a function of $\lambda$: $$d\mathcal{O}$$

In [1]:
from matplotlib import pyplot as plt
import numpy as np, scipy as sp, seaborn as sns
from ipywidgets import interact, fixed
%matplotlib inline

ImportError: dlopen(/Users/mglerner/anaconda3/lib/python3.6/site-packages/matplotlib/ft2font.cpython-36m-darwin.so, 2): Library not loaded: @rpath/libpng16.16.dylib
  Referenced from: /Users/mglerner/anaconda3/lib/libfreetype.6.dylib
  Reason: Incompatible library version: libfreetype.6.dylib requires version 51.0.0 or later, but libpng16.16.dylib provides version 49.0.0